In [ ]:
# 1.1 extract first sentences of what people are saying after they enter

import glob
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
blocklist.append('Meltdown')
blocklist.append('14')

first_sentences = []

outfile = open('xxxchatters_entrance_first_words.txt', 'w')

def find_post(lines_list, a):
    for s in lines_list:
        if s[0] is '[':
            if s.find("<") is not -1:
                time = s[s.find("[")+1:s.find("]")]
                author = s[s.find("<")+1:s.find(">")]
                sentence = s[s.find(">")+2:]
                if a == author and sentence not in first_sentences:
                    first_sentences.append(sentence)
                    out = time + ", " + s
                    outfile.write(out + "\n")
                    print(out)
                    return

for file in glob.glob("xxxchatters_logs/subset/*.log"):
    lines = open(file, 'r', encoding='utf-8').readlines()
    index = 0
    for line in lines:
        if not in_blocklist(blocklist, line.strip()): # blocked?
            if ") Quit (" in line or ") has left " in line:
                time = line[line.find("[")+1:line.find("]")]
                author = line[line.find(" * ")+3:line.find(" (")]
                sentence = line[line.find(">")+2:]
                
                #print("left: " + time + ", " + author)# + ", " + sentence)
            if ") has joined " in line:
                time = line[line.find("[")+1:line.find("]")]
                author = line[line.find(" * ")+3:line.find(" (")]
                #print("joined: " + time + ", " + author)
                find_post(lines[index:], author)
                
        index += 1
        
outfile.close()

In [227]:
# 1.2 look at word frequency

import glob
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
#blocklist.append('Meltdown')
#blocklist.append('14')
blocklist.append('type it in the main room or a pm to the')


full_lines = []

for file in glob.glob("xxxchatters_logs/*.log"):
    lines = open(file, 'r', encoding='utf-8').readlines()
    
    for line in lines:
        if not in_blocklist(blocklist, line.strip()): # blocked?
            if line[0] is '[':
                full_lines.append(line)
            

lines = open("1006/410.txt", 'r').readlines()
for line in lines:
    if ':' in line:
        #author = line[:line.find(":")]
        #authors.append(author)
        sentence = line[line.find(":"):]
        full_lines.append(line)

lines = open("1006/501.txt", 'r').readlines()
for s in lines:
    if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
        time = s[s.find("[")+1:s.find("]")]
        author = s[s.find("<")+1:s.find(">")]
        sentence = s[s.find(">")+2:]
        #authors.append(author)
        #sentences.append(sentence)
        full_lines.append(s)
print(len(full_lines))

219773


In [233]:
authors = {}
posts = set()
all_posts = []

for s in full_lines:
    if s.find("<") is not -1:
        time = s[s.find("[")+1:s.find("]")]
        author = s[s.find("<")+1:s.find(">")]
        sentence = s[s.find(">")+2:]
        all_posts.append(sentence.strip().lower())
        posts.add(sentence.strip().lower())
        if author not in authors: # save things
            authors[author] = [] # init list for author
        authors[author].append(sentence.strip()) 
    
print('Different Authors: ' + str(len(authors)) + '\n')
print('Lines: ' + str(len(all_posts)) + '\n')

Different Authors: 5819

Lines: 219460



In [ ]:
djmercury = authors['+DJ`Mercury']

In [234]:
import operator
import string
word_counts = dict()
translator = str.maketrans('', '', string.punctuation)

for post in posts:
    word_list = post.split()
    for word in word_list:
        word = word.replace('\x02', '')
        word = word.replace('\x03', '')
        word = word.replace('.', '')
        word = word.translate(translator)
        if len(word) < 1:
            continue
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

sorted_word_counts = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
# prepare list of top n words

n = 2500
print(sorted_word_counts[:n])
top_n = []
for t in sorted_word_counts[:n]:
    top_n.append(t[0])
    
print(len(top_n))

In [73]:
# only posts longer than 20 and only high frequent words

def contains_all(_list, _words):
    for word in _words:
        if word not in _list:
            return False
    
    return True;

file = open("1006/only_top300_and_longer_than_20.txt", "w")

for post in posts:
    words = post.split()
    if contains_all(top_n, words):
        if(len(post) > 20):
            file.write(post + '\n')
            #print(post)
            
file.close()

In [244]:
# removes all non-frequent words from long ones
import random

def remove_non_frequent(_list, post, replace=True):
    good_words = []
    for word in post.split():
        if word in _list:
            good_words.append(word)
        else:
            if replace:
                replacement = ""
                for char in word:
                    replacement += "-"
                good_words.append(replacement)
    
    return " ".join(good_words)

file = open("1006/7_named_removed_top2500_and_longer_than_80_before_replaced_unique_lines.txt", "w")


author_probs = get_author_probs(len(posts))
for i in range(500):
#for post in posts:
    post = list(posts)[i]
    if len(post) > 0:
        perc = i / float(500)
        n = 50000 * (1 - perc) 
        top_n = []
        for t in sorted_word_counts[:int(n)]:
            top_n.append(t[0])
        stripped_post = remove_non_frequent(top_n, post.strip().lower(), True)
        if len(stripped_post) > 0:
            a = author_probs[i]
            try:
                if a == '+DJ`Mercury':
                    file.write(a + ": " + random.choice(authors['+DJ`Mercury']) + '\n')
                else:
                    file.write(a + ": " + stripped_post + '\n')
            except:
                pass
            #print(stripped_post)
        
file.close()
    

[('@erotic_kitty', 67), ('carrol', 54), ('+DJ`Mercury', 39), ('Louise', 33), ('@erotic_kitty{S}', 27), ('Valkyrie', 23), ('Threeleggedcat', 20), ('@saffron{WH}', 18), ('PlayfulBBC', 18), ('WhoGiveSaDamn', 15), ('Silver_haired`Fox', 14), ('sweet_teresa', 10), ('Woman', 8), ('Jothom', 8), ('Cruel`Intentions', 7), ('guynextdoor', 7), ('Anastatia', 7), ('^fran', 7), ('Sanger', 7), ('Jaems', 6), ('+DJ`South', 6), ('JFetish', 5), ('FunkyBoogieKing', 5), ('gracie', 4), ('@DJ`liltech', 4), ('Handyman', 4), ('MeanMark1', 4), ('Athena', 3), ('saffron{WH}', 3), ('TricksyM', 3)]
returning: ['+DJ`Mercury' 'Woman' 'Athena' ... '+DJ`Mercury' 'Louise' '+DJ`Mercury']


In [192]:
# extract authors probability
import numpy as np

def get_author_probs(count):

    author_probability = {}

    lines = open("1006/410.txt", 'r').readlines()
    for line in lines:
        if ':' in line:
            author = line[:line.find(":")]
            if author in author_probability.keys():
                author_probability[author] += 1
            else:
                author_probability[author] = 1

    lines = open("1006/501.txt", 'r').readlines()
    for s in lines:
        if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
            time = s[s.find("[")+1:s.find("]")]
            author = s[s.find("<")+1:s.find(">")]
            sentence = s[s.find(">")+2:]
            if not '[' in author:
                if author in author_probability.keys():
                    author_probability[author] += 1
                else:
                    author_probability[author] = 1
    author_probability_counts = sorted(author_probability.items(), key=operator.itemgetter(1), reverse=True)
    top_authors = 30;
    print(author_probability_counts[:top_authors])

    sum_sentences = 0
    for v in author_probability_counts[:top_authors]:
        sum_sentences += v[1]

    #print(sum_sentences)
    author_probability_final = {}
    author_list = []
    prob_list = []

    for v in author_probability_counts[:top_authors]:
        author_probability_final[v[0]] = v[1]/float(sum_sentences)
        author_list.append(v[0])
        prob_list.append(v[1]/float(sum_sentences))

    #print(author_probability_final)

    final_autor_list = np.random.choice(
      author_list, 
      count,
      p=prob_list
    )

    print("returning: " + str(final_autor_list))
    return final_autor_list

In [231]:
authors = []
sentences = []

def remove_non_frequent2(_list, post, replace=True):
    good_words = []
    for word in post.split():
        if word in _list:
            good_words.append(word)
        else:
            #print(word)
            if replace:
                replacement = ""
                for char in word:
                    replacement += "-"
                good_words.append(replacement)
    
    return " ".join(good_words)

lines = open("1006/410.txt", 'r').readlines()
for line in lines:
    if ':' in line:
        author = line[:line.find(":")]
        authors.append(author)
        sentence = line[line.find(":"):]
        sentences.append(sentence)

lines = open("1006/501.txt", 'r').readlines()
for s in lines:
    if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
        time = s[s.find("[")+1:s.find("]")]
        author = s[s.find("<")+1:s.find(">")]
        sentence = s[s.find(">")+2:]
        if not '[' in author:
            authors.append(author)
            sentences.append(sentence)
            

outfile = open('1006/5_gradient_text_410_501_2.txt', 'w')
            
for i in range(len(authors)):
    perc = i/float(len(authors))
    a = authors[i]
    s = sentences[i]
    
    n = 100000 * (1 - perc) 
    top_n = []
    for t in sorted_word_counts[:int(n)]:
        top_n.append(t[0])
    #print(len(top_n))
    stripped_post = remove_non_frequent2(top_n, s.strip().lower(), True)
    
    outfile.write(a + ": " + stripped_post + '\n')

outfile.close()